[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

El formato PDF es un [estándar internacional](https://www.iso.org/standard/51502.html) para la creación de documentos que pueden ser desplegados o impresos de forma idéntica independientemente del dispositivo o aplicación que acceda a dichos documentos.

El énfasis del formato PDF reside en la correcta legibilidad de la información para los seres humanos, poniendo en segundo término la estructura y orden de la información que contiene.


## Creación de documentos PDF a partir de HTML con *wkhtmltopdf*.

El proyecto https://wkhtmltopdf.org/ ofrece una herramienta capaz de transformar código HTML en documentos PDF. Dicha herramienta se ejecuta directamente como un comando del sistema con la sintaxis.

```
wkhtmltopdf <archivo HTML de origen> <archivo PDF resultante> <parámetros>
```

### Restricciones. 

El paquete *wkhtmltopdf* requiere del uso de un servidor X y la biblioteca Qt. En el caso de las instalaciones en sistemas Windows, los binarios incluyen lo necesario para poder ejecutar la herramienta desde una terminal. Del mismo modo, los sistemas de escritorio de Linux también cuentan con un entrono gráfico gestionado por un servidor X. Sin embargo, es muy común que un servidor Linux no cuente con un servidor X instalado.

Para que *pdfkit* pueda funcionar, es necesario que *wkhtmltopdf* se encuentre en una ruta conocida, por lo que en el caso de Windows, es necesario añadir esta ruta a la variable de entorno *PATH*.

## Instalación de *wkhtmltopdf* junto con un servidor virtual de X en CentOS 7.

La máquina virtual que fue descargada del sitio http://pythonista.io no cuenta con un servidor X, por lo que es necesario instalarlo al igual que el paquete *wkhtmltopdf*.

**Nota:**
Este ejercicio representa el caso menos favorable. Si se utiliza esta herramienta desde un sistema Windows o desde un entorno gráfico en Linux, el uso de esta herramienta es mucho más simple.

### Instalación y configuración del servidor X virtual.
El servidor X virtual o *xvfb* por las siglas en inglés de "X Virtual FrameBuffer" es una variante de un servidor X que realiza las mismas opraciones que uno convencional, pero no se despliega en pantalla. Se utiliza para realizar opraciones que requieren de un entrono gráfico en un sistema GNU/Linux.

#### Instalación del paquete.

En este caso se utilizará el gestor de paquetes, *yum*.

In [1]:
!sudo yum install xorg-x11-server-Xvfb -y

Loaded plugins: fastestmirror
Determining fastest mirrors
epel/x86_64/metalink                                     |  16 kB     00:00     
 * base: repos-tx.psychz.net
 * epel: mirror.compevo.com
 * extras: repos.dfw.quadranet.com
 * updates: mirrors.usc.edu
base                                                     | 3.6 kB     00:00     
epel                                                     | 5.4 kB     00:00     
extras                                                   | 3.4 kB     00:00     
mongodb-org-4.0                                          | 2.5 kB     00:00     
updates                                                  | 3.4 kB     00:00     
(1/8): base/7/x86_64/group_gz                              | 166 kB   00:00     
(2/8): epel/x86_64/group_gz                                |  88 kB   00:01     
(3/8): epel/x86_64/updateinfo                              | 998 kB   00:01     
(4/8): base/7/x86_64/primary_db                            | 6.0 MB   00:02     
(5/8): extra

  Verifying  : libfontenc-1.1.3-3.el7.x86_64                              11/13 
  Verifying  : xorg-x11-server-common-1.20.1-5.6.el7_6.x86_64             12/13 
  Verifying  : 1:xorg-x11-xauth-1.0.9-1.el7.x86_64                        13/13 

Installed:
  xorg-x11-server-Xvfb.x86_64 0:1.20.1-5.6.el7_6                                

Dependency Installed:
  libICE.x86_64 0:1.0.9-9.el7                                                   
  libSM.x86_64 0:1.2.2-2.el7                                                    
  libXdmcp.x86_64 0:1.1.2-6.el7                                                 
  libXfont2.x86_64 0:2.0.3-1.el7                                                
  libXmu.x86_64 0:1.1.2-2.el7                                                   
  libXt.x86_64 0:1.1.5-3.el7                                                    
  libfontenc.x86_64 0:1.1.3-3.el7                                               
  libxkbfile.x86_64 0:1.0.9-3.el7                                         

#### Creación del archivo de definición de servicio.
Una vez instalado, se define el servicio añadiendo el archivo [*src/xvfb.service*](src/xvfb.service) en */etc/systemd/system/* como superusuario.

Dicho archivo contiene el siguiente código:

``` bash
[Unit]
Description=Virtual Frame Buffer X Server
After=network.target

[Service]
ExecStart=/usr/bin/Xvfb :42 -screen 0 1024x768x24 -ac +extension GLX +render -noreset

[Install]
WantedBy=multi-user.target
```

In [2]:
!sudo cp src/xvfb.service /etc/systemd/system/

#### Habilitación y puesta en marcha del servicio.
CentOS 7 utiliza el comando *systemctl* para gestionar servicios.

In [3]:
!sudo systemctl enable xvfb

Created symlink from /etc/systemd/system/multi-user.target.wants/xvfb.service to /etc/systemd/system/xvfb.service.


In [4]:
!sudo systemctl start xvfb

Para comprobar que el servidor está corriendo ejecute lo siguiente:

In [5]:
!sudo systemctl status xvfb

● xvfb.service - Virtual Frame Buffer X Server
   Loaded: loaded (/etc/systemd/system/xvfb.service; enabled; vendor preset: disabled)
   Active: active (running) since Thu 2019-08-29 16:01:53 CDT; 10s ago
 Main PID: 3991 (Xvfb)
   CGroup: /system.slice/xvfb.service
           └─3991 /usr/bin/Xvfb :42 -screen 0 1024x768x24 -ac +extension GLX ...

Aug 29 16:01:53 pythonista.localdomain systemd[1]: Started Virtual Frame Buff...
Hint: Some lines were ellipsized, use -l to show in full.


**Nota:** Este procedimiento sólo se requiere hace una vez. Una vez instalado, configurado y habilitado, el servicio iniciará junto con el servidor.

### Instalación y configuración de *wkhtmltopdf*.

#### Instalación del paquete con *yum*.

In [6]:
!sudo yum install wkhtmltopdf -y

Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: repos-tx.psychz.net
 * epel: mirror.compevo.com
 * extras: repos.dfw.quadranet.com
 * updates: mirrors.usc.edu
Resolving Dependencies
--> Running transaction check
---> Package wkhtmltopdf.x86_64 0:0.12.1-1.el7 will be installed
--> Processing Dependency: libQtCore.so.4()(64bit) for package: wkhtmltopdf-0.12.1-1.el7.x86_64
--> Processing Dependency: libQtGui.so.4()(64bit) for package: wkhtmltopdf-0.12.1-1.el7.x86_64
--> Processing Dependency: libQtNetwork.so.4()(64bit) for package: wkhtmltopdf-0.12.1-1.el7.x86_64
--> Processing Dependency: libQtSvg.so.4()(64bit) for package: wkhtmltopdf-0.12.1-1.el7.x86_64
--> Processing Dependency: libQtWebKit.so.4()(64bit) for package: wkhtmltopdf-0.12.1-1.el7.x86_64
--> Processing Dependency: libQtXmlPatterns.so.4()(64bit) for package: wkhtmltopdf-0.12.1-1.el7.x86_64
--> Running transaction check
---> Package qt.x86_64 1:4.8.7-3.el7_6 will be installed
--> Processing D

 libvisual                  x86_64     0.4.0-16.el7           base        138 k
 libvorbis                  x86_64     1:1.3.3-8.el7.1        base        204 k
 libwebp                    x86_64     0.3.0-7.el7            base        170 k
 mozilla-filesystem         x86_64     1.9-11.el7             base        5.3 k
 orc                        x86_64     0.4.26-1.el7           base        166 k
 pango                      x86_64     1.42.4-2.el7_6         updates     280 k
 pciutils                   x86_64     3.5.1-3.el7            base         93 k
 qt                         x86_64     1:4.8.7-3.el7_6        updates     4.6 M
 qt-settings                noarch     19-23.8.el7.centos     base         17 k
 qt-x11                     x86_64     1:4.8.7-3.el7_6        updates      13 M
 qtwebkit                   x86_64     2.3.4-6.el7            epel         10 M
 xml-common                 noarch     0.6.3-39.el7           base         26 k

Transaction Summary
Install  1 Package 

  Verifying  : orc-0.4.26-1.el7.x86_64                                    20/33 
  Verifying  : libvisual-0.4.0-16.el7.x86_64                              21/33 
  Verifying  : harfbuzz-1.7.5-2.el7.x86_64                                22/33 
  Verifying  : qtwebkit-2.3.4-6.el7.x86_64                                23/33 
  Verifying  : 1:qt-x11-4.8.7-3.el7_6.x86_64                              24/33 
  Verifying  : 2:libogg-1.3.0-7.el7.x86_64                                25/33 
  Verifying  : libXrandr-1.5.1-2.el7.x86_64                               26/33 
  Verifying  : fribidi-1.0.2-1.el7.x86_64                                 27/33 
  Verifying  : jbigkit-libs-2.0-11.el7.x86_64                             28/33 
  Verifying  : xml-common-0.6.3-39.el7.noarch                             29/33 
  Verifying  : graphite2-1.3.10-1.el7_3.x86_64                            30/33 
  Verifying  : iso-codes-3.46-2.el7.noarch                                31/33 
  Verifying  : gstreamer-too

#### Creación de un script que permita acceder al servidor virtual.

Es necesario que wkthmltopdf pueda acceder al servidor virtual, por lo que se debe de crear un script que incluya al comando *xvfb-run* el cual le indica al sistema que la instrucción hará uso de un servidor X virtual.

El script [src/wkhtmltopdf.sh](src/wkhtmltopdf.sh) contiene el siguiente código:

``` bash
#!/bin/bash
xvfb-run -a --server-args="-screen 0, 1280x800x24" /bin/wkhtmltopdf -q $*
```

El script en cuestión debe de contar con permisos de ejecución, lo cual se realiza con el siguiente comando. 

In [7]:
!chmod +x src/wkhtmltopdf.sh

#### Ejecución del script.

**Nota:** Este es un ejemplo de ejecución en la línea de comandos. No es código en Python. 

La siguiente celda ejecutará el script [*src/wkhtmltopdf.sh*]([src/wkhtmltopdf.sh]) 
* Se conectará a *http://google/com*.
* El contenido será convertido a PDF y guarado en [*google.pdf*](google.pdf).

In [8]:
!src/wkhtmltopdf.sh http://google.com google.pdf

## Instalación de *wkhtmltopdf* en Windows.

En el caso de Windows sólo es necesario [descargar del sitio](https://wkhtmltopdf.org/downloads.html) la versión *MSVC 2015* y ejecutar el instalador.

Por lo general, el ejecutable se instala en *C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe*

**Ejemplo:**

La siguiente celda ejecutará *wkhtmltopdf.exe* en la ruta indicada. 
* Se conectará a *http://google/com*.
* El contenido será convertido a PDF y guarado en [*google.pdf*](google.pdf).

In [ ]:
!"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf" "https://google.com" "google.pdf"

## El paquete *pdfkit*.

El paquete [*pfdkit*](https://pypi.org/project/pdfkit/#description) es una implementación de *wkhtmltopdf* para Python.

In [9]:
!pip install pdfkit

You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import pdfkit

### Funciones de *pdfkit* dependiendo la fuente de los datos.

Es posible tranformar documentos HTML desde diversas fuentes.

#### Si el código HTML es ingresado como una cadena de caracteres se utiliza la función *pdfkit.fromstring()* con la siguiente  sintaxis:
```
pdfkit.fromstring(<objeto tipo str>, '<ruta del archivo resultante>', <argumentos>)
```

#### Si el código HTML proviene de un archivo local, se utiliza la función *pdfkit.fromfile()* con la siguiente  sintaxis:
```
pdfkit.fromfile('<ruta>', '<ruta del archivo resultante>', <argumentos>)
```

#### Si el código HTML proviene de un recurso en línea, se utiliza la función *pdfkit.fromulr()* con la siguiente  sintaxis:
```
pdfkit.fromurl('<URL>', '<ruta del archivo resultante>', <argumentos>)
```

### El objeto *pdfkit.configuration*.
Para poder utilizar *pdfkit* es necesario que *wkhtmltopdf* esté instalado y que la ruta donde se encuentra el ejecutable sea conocida.

El objeto *pdfkit.configuration* permite ingresar diversos parámetros de configuración y en este caso se puede indicar dónde se encuentra el archivo ejecutable de *wkhtmltopdf* con la siguiente sintaxis:

```python
pdfkit.configuration(wkhtmltopdf='<ruta>')
```
En caso de que el ejecutable de *wkhtmltopdf* sea accesible desde el sistema, no es necesario utilizar este objeto.

**Ejemplo:** 

En este caso, el ejecutable que utilizará *pdfkit* corresponde al script localizado en [src/wkhtmltopdf.sh](src/wkhtmltopdf.sh)

In [11]:
conf = pdfkit.configuration(wkhtmltopdf='src/wkhtmltopdf.sh')

En caso de haber instalado *wkhtmltopdf* en Windows, la configuración sería:

In [ ]:
conf = pdfkit.configuration(wkhtmltopdf='C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe')

**Ejemplos:**

Se convertirán archivos HTML localizados en archivos locales y en línea mediante *pdfkit*.

En este caso se convertirá el contenido de [html/menu.html](html/menu.html) y se guardará en [menu.pdf](menu.pdf).

In [12]:
pdfkit.from_file('html/menu.html', 'menu.pdf', configuration=conf)

True

En este caso se convertirá el contenido de https://pythonista.io/cursos/py101/ciclos-con-while-e-interrupciones-de-flujo y se guardará en [ciclos.pdf](ciclos.pdf).

In [13]:
pdfkit.from_url('https://pythonista.io/cursos/py101/ciclos-con-while-e-interrupciones-de-flujo',
                'ciclos.pdf', configuration=conf)

True

### El parámetro *options*.

Es posible modificar el tipo y orientación de la página, el tamaño de los márgenes y algunas otras características del documento PDF resultante mediante algunas opciones de *wkhtmltopdf*, las cuales pueden ser consultadas en https://github.com/worlduniting/bookshop/wiki/wkhtmltopdf-options

En el caso de *pdfkit* estas opciones deben de estar contenidas en un objeto tipo *dict*.

**Ejemplo:**

Se creará nuevamente el archivo [menu.pdf](menu.pdf) con algunos ajustes a los márgenes e indicando el tipo de tamano de papel.

In [14]:
options = {
    'page-size': 'Letter',
    'margin-top': '0.75in',
    'margin-right': '0.75in',
    'margin-bottom': '0.75in',
    'margin-left': '0.75in',
    'encoding': "UTF-8"
}

In [15]:
pdfkit.from_file('html/menu.html', 'menu.pdf', configuration=conf, options=options)

True

## Lectura de archivos PDF.

La extracción de datos a partir de archivos PDF no es simple y en muchos casos tampoco es infalible. Es por ello por lo que se detallará el uso de las siguientes herramientas.

* [*pdfminer.six*](https://github.com/pdfminer/pdfminer.six).
* [*pdftotext*](https://github.com/jalan/pdftotext).
* [*PyPDF2*](https://mstamy2.github.io/PyPDF2/).

### El paquete *pdfminer.six*.

El paquete *pdfminer.six* es una implementación compatible con Python 3 de *pdfminer*. Es una herramienta especializada para la extracción de texto muy popular. 

Tiene la  ventaja de acceder  a porciones específicas de un documento PDF y gestionarlas en memoria.

Los paquetes *pdfminer* y *pdfminer.six* cuentan con múltiples clases especializadas en gestionar, seleccionar, obtener, interpretar y convertir porciones específicas de un documento PDF.

Para mayor información consultar https://pdfminer-docs.readthedocs.io/

#### Extracción de un documento PDF con *pdfminer.six*.

A continuacion se creará una función capaz de extraer el texto de las pág
inas indicadas desde un archivo PDF.

* Se instala el paquete *pdfminer.six*.

In [16]:
! pip install pdfminer.six

    100% |████████████████████████████████| 5.6MB 1.0MB/s ta 0:00:011.6MB/s eta 0:00:01
    100% |████████████████████████████████| 9.7MB 1.7MB/s eta 0:00:01��█████▏                | 4.6MB 23.7MB/s eta 0:00:01    63% |████████████████████▍           | 6.2MB 19.5MB/s eta 0:00:01     | 7.0MB 43.0MB/s eta 0:00:0179% |█████████████████████████▌      | 7.7MB 33.7MB/s eta 0:00:01 eta 0:00:01████████████▍   | 8.6MB 20.4MB/s eta 0:00:01   93% |█████████████████████████████▉  | 9.0MB 26.8MB/s eta 0:00:01█████▎| 9.5MB 4.4MB/s eta 0:00:01
You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


* La clase *io.StringIO* del paquete *io* de Python permite acceder a un recurso en el disco y guardarlo en memoria. De este modo podemos guardar porciones del archivo PDF en un *buffer*.

In [17]:
from io import StringIO

In [19]:
!pip install chardet

    100% |████████████████████████████████| 143kB 2.3MB/s ta 0:00:01
You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


El módulo *pdfminer.pdfinterp* contiene a las clases:

* *PDFResourceManager*, cuyas instancias son capaces de gestionar el flujo de recursos de un documento PDF a los que *pdfminer* accede. En este caso, se encargará de  gestionar el flujo entre un archivo y el buffer en memoria.
* *PDFPageInterpreter*, cuyas instancias tienen la capacidad de acceder e interpetar el contenido de un recurso proveniente de un archivo PDF.

In [20]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

* El módulo *pdfminer.converter* contiene varias clases capaces de convertir contenidos específicos de in documento PDF, talkes como XML, HTML o texto.

* En este caso, se utilizará la clase *TextConverter*.

In [21]:
from pdfminer.converter import TextConverter

Los archivos PDF incluyen por lo general una plantilla (layout) específica del documento que puede definir el tamaño de página, orientacion, numeración, márgenes, etc.

* El módulo *pdfminer.layout* permite acceder a la información de estas plantillas y la clase *pdfminer.layout.LAParams*, se encarga de obtener los parámetros definidos en la plantilla de un documento PDF.

In [22]:
from pdfminer.layout import LAParams

Los documentos PDF está compuestos por páginas cuyas características están definidas en los parámetros de la plantilla correspondiente.

* La clase *pdfminer.pdfpage.PDFPage* crea objetos que emulan a una página del documento PDF.

In [23]:
from pdfminer.pdfpage import PDFPage

* La función *convierte()* permite extraer el texto de las páginas enumeradas dentro de un objeto iterable que se ingresa como el argumento *paginas* a partir del archivo definido en el parámetro *ruta*. En caso de no indicarlos, extraerá el texto de todas las páginas del documento.

In [25]:
def convierte(ruta, paginas=None):
    if not paginas:
        numero_paginas = set()
    else:
        numero_paginas = set(paginas)
    salida = StringIO()
    gestor = PDFResourceManager()
    convertidor = TextConverter(gestor, salida, laparams=LAParams())
    interprete = PDFPageInterpreter(gestor, convertidor)

    archivo = open(ruta, 'rb')
    for pagina in PDFPage.get_pages(archivo, numero_paginas):
        interprete.process_page(pagina)
    archivo.close()
    convertidor.close()
    texto = salida.getvalue()
    salida.close
    return texto

**Ejemplos:**

* Se obtendrá el texto de la segunda página del archivo [*menu.pdf*](menu.pdf) que fue creado previamente.

In [26]:
convierte('menu.pdf', paginas=[1])

'Nuestras gringas muy mexicanas\nNuestras gringas muy mexicanas\n\nPavo al pastor sobre tortillas de harina hechas a mano y queso\nmenonita. Insuperable.\n\nPepitos de arrachera\nPepitos de arrachera\n\nUna explosión de sabores con nuestros finos cortes seleccionados\ny macerados\npor nuestro chef, y servidos en pan de nuestro horno.\n\nHuachinango a la sal\nHuachinango a la sal\n\nNuestro pescado es traído diariamente desde Veracruz hasta su\nmesa.\n\nPostres\n\nPan de pulque\nPan de pulque\n\nNuestro famoso pan de pulque hecho con el néctar traído de\nHidalgo.\nNo olvide pedir para llevar.\n\nNieve de leche quemada con tuna\nNieve de leche quemada con tuna\n\nDisfrute este tradicional postre oaxaqueño servido con las soletas\npreparadas\npor nosotros.\n\nSorbete de rosas y vino blanco espumoso\nSorbete de rosas y vino blanco espumoso\n\nNuestro postre más delicado, para paladares exigentes.\n\nCrepas al momento\nCrepas al momento\n\nPermítanos preparar su crepa frente a usted para qu

* Se obtendrá el texto completo del archivo [*menu.pdf*](menu.pdf).

In [27]:
convierte('menu.pdf')

'Restaurante-Bar\n\n"La Fonda de la Marquesa"\n"La Fonda de la Marquesa"\n\nEl lugar de los amigos y cómplices desde 1987.\n\nInicio   Menú   Equipo\n\nNuestro Menú\n\nEntradas\n\nEnsalada mixta\nEnsalada mixta\n\nPreparada con varios tipos de lechuga, cebolla, jitomate y aderezo\nde la casa.\n\nCorazones de alcachofa al vapor\nCorazones de alcachofa al vapor\nServidos en un espejo de vinagreta.\n\nEmpanadas argentinas (3)\nEmpanadas argentinas (3)\n\nNuestras tradicionales empanadas rellenas de carne, queso o\nchistorra a escoger.\n\nSashimi de pepino\nSashimi de pepino\n\nUna fresca opción con nuestros cortes extra finos y la delicadeza\nde la salsa de soya.\n\nSopas\n\nConsomé de la Condesa\nConsomé de la Condesa\n\nNuestra sazón especial para un platillo muy tradicional.\n\nJugo de carne\nJugo de carne\n\nHecho con la más fina carne de Sonora.\n\nSopa de lentejas\nSopa de lentejas\n\nCon plátano macho y tocino.\n\nSopa de fideos\nSopa de fideos\n\nGuisado con nuestro consomé especi

### El paquete *pdftotext*.

Este paquete es una implementación de la biblioteca [*poppler*](https://poppler.freedesktop.org/), perteneciente a [XpdfReader](https://www.xpdfreader.com/). Este paquete corre en Linux, UNIX y MacOS X.

In [28]:
!sudo yum install poppler-cpp-devel -y

Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: repos-tx.psychz.net
 * epel: mirror.compevo.com
 * extras: repos.dfw.quadranet.com
 * updates: mirrors.usc.edu
Resolving Dependencies
--> Running transaction check
---> Package poppler-cpp-devel.x86_64 0:0.26.5-20.el7 will be installed
--> Processing Dependency: poppler-devel(x86-64) = 0.26.5-20.el7 for package: poppler-cpp-devel-0.26.5-20.el7.x86_64
--> Processing Dependency: poppler-cpp(x86-64) = 0.26.5-20.el7 for package: poppler-cpp-devel-0.26.5-20.el7.x86_64
--> Processing Dependency: pkgconfig(poppler) = 0.26.5 for package: poppler-cpp-devel-0.26.5-20.el7.x86_64
--> Processing Dependency: libpoppler-cpp.so.10()(64bit) for package: poppler-cpp-devel-0.26.5-20.el7.x86_64
--> Running transaction check
---> Package poppler-cpp.x86_64 0:0.26.5-20.el7 will be installed
--> Processing Dependency: poppler(x86-64) = 0.26.5-20.el7 for package: poppler-cpp-0.26.5-20.el7.x86_64
--> Processing Dependency: libpop

In [29]:
!pip install pdftotext

    100% |████████████████████████████████| 122kB 1.5MB/s ta 0:00:01
  Stored in directory: /home/oi/.cache/pip/wheels/43/14/6c/51e06da6a5ba1b2d9169b96d8c9cc30ae96e6ba34be0d4bd6c
Successfully built pdftotext
You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [30]:
import pdftotext

La clase *pdftotext.PDF* permite crear un objeto iterador que contienen el texto extraido de cada página de un documento PDF.

In [31]:
with open("menu.pdf", "rb") as archivo:
    pdf = pdftotext.PDF(archivo)

In [32]:
type(pdf)

pdftotext.PDF

In [33]:
for page in pdf:
    print(page)

                               Restaurante-Bar
                      "La Fonda de la Marquesa"
              El lugar de los amigos y cómplices desde 1987.
                              Inicio        Menú        Equipo
                         Nuestro Menú
                                 Entradas
Ensalada mixta
    Preparada con varios tipos de lechuga, cebolla, jitomate y aderezo  $65.00
    de la casa.
Corazones de alcachofa al vapor
                                                                       $125.00
    Servidos en un espejo de vinagreta.
Empanadas argentinas (3)
    Nuestras tradicionales empanadas rellenas de carne, queso o        $130.00
    chistorra a escoger.
Sashimi de pepino
    Una fresca opción con nuestros cortes extra finos y la delicadeza  $150.00
    de la salsa de soya.
                                     Sopas
Consomé de la Condesa
                                                                        $85.00
    Nuestra sazón especial para un platillo m

### El paquete *PyPDF2*.

Existen diversos proyectos enfocados a la gestión de archivos en formato PDF. El paquete *PyPDF2* permite no sólo leer sino escribir, concatenar y modificar archivos PDF. 

Para los fines de esta sección sólo se utilizará la clase *PyPDF2.PdfFileReader* para leer el contenido de un archivo PDF.

Para saber mas sobre este paquete puede consultar en https://pythonhosted.org/PyPDF2/

In [34]:
!pip install PyPDF2

    100% |████████████████████████████████| 81kB 942kB/s ta 0:00:01
  Stored in directory: /home/oi/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2
You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


#### La clase *PyPDF2.PdfFileReader*

Esta clase crea un objeto que posee atributos correspontientes a las características, estructuras y contenidos de un archivco PDF. Para crear un objeto a partir de esta clase su tiliza la siguiente sintaxis:

```
PyPDF2.PdfFileReader('<ruta del archivo>')
```
Aún cuando el paquete es muy versatil, en muchos casos no es tan eficiente para extraer texto como las herramientas mencionadas anteriormente.

In [35]:
from PyPDF2 import PdfFileReader

In [36]:
with open('menu.pdf', 'rb') as archivo:
    pdf = PdfFileReader(archivo)
    print('Información del documento:', pdf.documentInfo)
    print('Número de páginas:', pdf.numPages)
    pagina = pdf.getPage(0)
    print('Contenido de la pagina 1:\n', pagina.extractText())

Información del documento: {'/Title': '', '/Creator': 'wkhtmltopdf 0.12.1', '/Producer': 'Qt 4.8.7', '/CreationDate': 'D:20190829213504'}
Número de páginas: 2
Contenido de la pagina 1:
 


<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2019.</p>